# Strömungsgeschwindigkeit über die Wassertiefe

Diese untenstehende Lösung visualisiert ein, **vertikales Geschwindigkeitsprofil in einer freien Strömung in Abhängigkeit verschiedener Parameter** und hilft z.B. 

1. besser ein generelles Systemverständnis aufzubauen,  
2. die Frage zu beantworten wie sich die tiefengemittelten Strömungsgeschwindigkeiten des Modells im Vergleich zu oberflächennahen Verhalten.


Hier wird die analytische/empirische Gleichung nach [1] für die Geschwindigkeit in Abhängigkeit der Entfernung von der Sohle ($v(z)$)gelöst:

$v(z) = s \cdot \sqrt{\frac{\tau_0}{\rho}} \cdot \ln(\frac{z}{k})$

mit den Variablen:

| Variable | Erklärung | Einheit |
|:---|:---|---|
| v  | Strömungsgeschwindigkeit | m/s |
| $z$ | Position über Sohle | m |
| $\tau_0$ | Wandschubspannung | N/m² |
| $\rho$ | Dichte des Fluides | kg/m³ |
| $k$ | Sohlrauheit | m |
| $s$ | Koeffizient (typisch 2,5) | - |

die Lösung erfolgt jeweils über alle $z$-Werte des Wasserstandes $h$

In [1]:
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
#plt.style.use('ggplot');

#from statsmodels.graphics import tsaplots # für korrelation, dekomposition etc
#import statsmodels.api as sm # für decomposition
from ipywidgets import IntSlider, FloatSlider
from ipywidgets import Layout, interact
lo = Layout(flex='1 1 0%', width='50%') # Layout (Breite etc der widgets)

def update_vplot(wl,tau_0,rho,k, s):
    z_all = np.linspace(0.0001,wl,wl*20) # geodätische Höhe von Sohle

    # löse für alle höhen/tiefen
    v = [s*np.sqrt(tau_0/rho)*np.log10(z/k) for z in z_all]
    vm = np.mean(v) # mittel
    # finde schnittpunkt 
    vi = 100
    for idx,vx in enumerate(v):
        if vi > abs(vx-vm):
            vi = abs(vx-vm)
            ix = idx
    zi = z_all[ix]
    
    # stelle grafisch dar
    fig, axs = plt.subplots(figsize=(14,8))
    axs.scatter(v,z_all,c=v,cmap='rainbow')
    axs.plot(v,z_all,linewidth=0.7,c='k',alpha=0.7)
    # plotte noch tiefenmittel und median
    axs.plot([vm,vm],[z_all[0],z_all[-1]], linewidth=1,c='grey',linestyle='--',label='depth averaged')
    # plotte schnittpunkt tiefenmittel mit Geschwindigkeitsprofil 
    # plotte rechte achse wie line aber normiert von 0-1(00)
    
    # markiere flächig was über mittel liegt
    plt.hlines(xmin=vm,xmax=v[-1],y=z_all[-1], linestyle = ':', color='r')
    
    # plotte box mit abstand v[-1] zu vm (absolut und relativ)
    # Fehlerbox
    bbox_props = dict(boxstyle="square", fc='w', ec="k", lw=1)
    axs.text(0.1, 0.93, "|v$_{of}$- v$_m$| = "+str(round(abs(v[-1]-vm),4))+" m/s \n"+
             "v$_{of}$- v$_m$/v$_{of}$ = "+str(  round(  (v[-1]-vm)/v[-1]*100)) +"%\n"+
             "z$_i$ = "+str(round(zi,2))+" m \n",
                        ha="left", va="top", rotation=0, size=15,
                        color='k', transform=axs.transAxes,
                        bbox=bbox_props);
    
    plt.xlabel('v [m/s]')
    plt.ylabel('z [m]')
    plt.title('h: '+str(wl)+' m | Tau$_0$: '+str(tau_0)+' | rho: '
              +str(rho)+ ' kg/m$^3$ | k: '+str(k) + ' m')
    #plt.grid()
    axs.set_xlim(0,0.5)
    # plotte sohle
    xl = axs.get_xlim()
    yl = axs.get_ylim()
    axs.fill([xl[0], xl[1], xl[1], xl[0] ],[ yl[0] , yl[0], 0, 0], facecolor="none", hatch="//", edgecolor="brown", linewidth=0.0)
    axs.axhline(0,linewidth=1, c='brown')
    axs.axhspan(0,wl ,alpha=.3,facecolor='lightblue')
    axs.set_ylim(yl[0],wl+(0.05*wl))

    plt.legend()
    plt.show()

    
# sliders
wis_wl = IntSlider(
    value=15,
    min=1,
    max=40,
    step=1,
    description='h [m]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_rho = FloatSlider(
    value=1000,
    min=800,
    max=1300,
    step=10,
    description='rho [kg/m^3]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_k = FloatSlider(
    value=0.001,
    min=0.001,
    max=0.5,
    step=0.001,
    description='k [m]', # friction coefficient
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_tau = FloatSlider(
    value=1,
    min=0.001,
    max=3,
    step=0.001,
    description=r'Tau_0 [F/L^2]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)
wfs_s = FloatSlider(
    value=2.5,
    min=0.01,
    max=4,
    step=0.01,
    description='s [-]',
    orientation='horizontal',
    continuous_update=False, # if false update is solely done when slider is "dropped"
    layout=lo)

interact(update_vplot,wl=wis_wl, tau_0 = wfs_tau, rho = wfs_rho, k = wfs_k, s = wfs_s);


interactive(children=(IntSlider(value=15, continuous_update=False, description='h [m]', layout=Layout(flex='1 …